# CPSC-483 Project 1: Outlier Analysis
## FA 2021 - Section 02
### Armanul Ambia, Iftekharul Islam
_______________________________________________________________________________________________________________________________

__Summary:__ This project contains 8 experiments where we analyze a given student attendance data file. This file contains student names along with the minutes they attended class for the first 5 weeks. Experiments 1 - 3 focus on reading the data into Python and then finding summary statistics, such as mean, median, and quartiles. Experiments 4 - 7 explore how we can use Tukey's Fences and standard deviations to find outliers in the data set. Experiment 8 compares our results using both methods and highlights the differences between these methods.

### __Experiment 1__ : Use the csv module to load the dataset and examine the contents of the first few rows.

In [42]:
import csv
participant_rdr = None
with open('participants.csv', newline='') as csvfile:
     participant_rdr = csv.DictReader(csvfile, delimiter=',', quotechar='|')
     print('{:<20}{:<10}{:<10}{:<10}{:<10}{:<10}'.format(
        *['Student Name','Week 1', 'Week 2', 'Week 3', 'Week 4', 'Week 5']))
     print()
     count = 0
     for row in participant_rdr:
         print('{:<20}{:<10}{:<10}{:<10}{:<10}{:<10}'.format(
              row['Student Name'],
              row['Week 1'], 
              row['Week 2'],
              row['Week 3'],
              row['Week 4'],
              row['Week 5']))
         count += 1
         if count == 5:
             break

Student Name        Week 1    Week 2    Week 3    Week 4    Week 5    

Adrian Ellison      77        154       4         170       175       
Ophelia Mcphee      179       151       164       173       171       
Yasir Fenton        180       47        164       168       169       
Benny Arias         180       152       161       170       170       
Tamara Cottrell     183       79        161       173       168       


In Experiment 1, we used the Python standard library's csv module to load in our data file, participants.csv. Afterwards, we used the reader function in the csv library to return an object that can iterate over all rows in the datafile. To examine the data, we used the for-loop to iterate and print out the first 5 rows in our csv file. This shows that the data has properly loaded and is ready for analysis in further experiments. 

For further optimization, we can use other methods in the csv library in order to avoid the count variable and break in our for loop. While this method works, there may be cleaner ways to use the iterator provided by the csv.DictReader function.
_____________________________________________________________________________________________________________________________

### __Experiment 2__ : Load the statistics module and use it to find the mean and median of Week 1’s data.

In [43]:
import statistics as st
week1 = []
with open('participants.csv', newline='') as csvfile:
     participant_rdr = csv.DictReader(csvfile, delimiter=',', quotechar='|')
     week1 = [int(row['Week 1']) for row in participant_rdr]
print('Mean of Week 1:', st.mean(week1))
print('Median of Week 1:', st.median(week1))

Mean of Week 1: 161
Median of Week 1: 175


### __Experiment 3__ : Find the quartiles for Week 1.

In [44]:
# using week1 list from experiment 2
# using st as the alias for imported stastics library from experiment 2
quartile_vals = st.quantiles(week1)
print(f"Quartile Values: {quartile_vals}")

Quartile Values: [174.0, 175.0, 179.0]


Quartiles are the numbers in a dataset that divide the dataset into 4 parts of approximately equal size. To find a quartile, one must first set all values in a dataset in non-decreasing order. Then, one must find the median. The median divides the set into two halves. The next two values are found by finding the median of these two halves.

The median of the lower half is the first quartile because it is greater than 25% of all datapoints in the dataset. Similarly, the second quartile is the median because it is greater than 50% of the datapoints in the dataset. The third quartile is the datapoint greater than 75% of the datapoints in the dataset.

With these in mind, we found the first, second, and third quartile values:
$$Q_1 = 174.0$$
$$Q_2 = 175.0$$
$$Q_3 = 179.0$$

These values appear to be very close together, but this is expected. The values in the dataset represent the total number of minutes each student spent in a Zoom conference call during each week. Because students attend and leave class at the around the same time, these numbers should be very close in range.

### __Experiment 4__ : Use the Tukey's Fences method with k = 1.5 to find the outliers in the Week 1 attendance data.


In [45]:
# using quartile_vals from experiment 3
q1 = quartile_vals[0]
median = quartile_vals[1]
q3 = quartile_vals[2]
iqr = q3-q1
k = 1.5
# the lower bound is equal to q1 - iqr * k according to Tukey's Fences
lb = q1 - iqr * k
# the upper bound is equal to q3 + iqr * k according to Tukey's Fences
ub = q3 + iqr * k

# populate the outliers dictionary with key: student name and value: week 1 value
outliers = {}
with open('participants.csv', newline='') as csvfile:
    participant_rdr = csv.DictReader(csvfile, delimiter=',', quotechar='|')
    for row in participant_rdr:
        name = row['Student Name']
        week_val =int(row['Week 1'])
        if week_val < lb or week_val > ub:
            outliers[name] = week_val
            
print(f"Interquartile Range: {iqr}")
print(f"Lower Bound: {lb}")
print(f"Upper Bound: {ub}")
print(f"Week 1 Outliers: {outliers}")

Interquartile Range: 5.0
Lower Bound: 166.5
Upper Bound: 186.5
Week 1 Outliers: {'Adrian Ellison': 77, 'Tayla Sparrow': 51, 'Owain Emerson': 9, 'Alaya Dickinson': 24}


In order to use the Tukey's Fences method, we had to use math along with our previous quartile values to find the Interquartile Range (IQR). From Experiment 3 we found that:
$$Q_1 =   174.0$$
$$Q_3 =   179.0$$
And thus, our IQR is:

$$IQR =   Q_3 - Q_1 = 179.0 - 174.0 = 5.0$$

The IQR for this data set is very close together. This makes sense since the quartile values are extremely close together, and this was further explored in the previous experiment.

Using this value, we used the scalar, __k = 1.5__, to calculate the lower and upper bounds:

$$LB = Q_1 - IQR * k = 174.0 - 5.0 * 1.5 = 166.5 $$
$$UB = Q_3 + IQR * k = 179.0 + 5.0 * 1.5 = 186.5 $$

Using these values, it is easy to see why these four students were returned by the algorithm. The lower bound obtained using this method is 166.5. The week 1 values for all four of these students is far below this lower bound and are clear outliers in the data. After examining the data, it is also apparent that no students were in class for over 186.5 minutes. 186.5 minutes is over 3 hours (180 min) in length. Since the class duration is 2 hours and 45 minutes it is expected that no students were in class for over 20 extra minutes. 


### __Experiment 5__ : Find the outliers in the Week 1 attendance data using the normal distribution and three standard deviations.


In [46]:
st_dev = st.stdev(week1)
mean = st.mean(week1)
# define lower bound as 3 standard deviations below the mean
lb = mean - 3*st_dev 
# define upper bound as 3 standard deviations above the mean
ub = mean + 3*st_dev  

# populate the outliers dictionary with key: student name and value: week 1 value
outliers = {}
with open('participants.csv', newline='') as csvfile:
    participant_rdr = csv.DictReader(csvfile, delimiter=',', quotechar='|')
    for row in participant_rdr:
        name = row['Student Name']
        week_val =int(row['Week 1'])
        if week_val < lb or week_val > ub:
            outliers[name] = week_val
            
print(f"Standard Deviation: {st_dev}")
print(f"Lower Bound: {lb}")
print(f"Upper Bound: {ub}")
print(f"Week 1 Outliers: {outliers}")

Standard Deviation: 45.02025687856262
Lower Bound: 25.939229364312155
Upper Bound: 296.06077063568785
Week 1 Outliers: {'Owain Emerson': 9, 'Alaya Dickinson': 24}


The first step in using this outlier method is to find the standard deviation. The statistics stdev function finds that:

$$ stdev = 45.0202567856262 $$

This number is very large. This means that one standard deviation contains 45 minutes of class time. This, however, does make sense since the data values are so close together. We saw with our close quartile values and small IQR that the data values are highly similar. Therefore, a large range of data points would be included through each standard deviation.

Afterwards, we used the mean value, __161__, to calculate the lower and upper bound:

$$LB = Mean - 3 * stdev = 161 - 3 * 45.0202567856262 = 25.939229364312155 $$
$$UB = Mean + 3 * stdev = 161 + 3 * 45.0202567856262 = 296.06077063568785 $$

For ease of interpretation:
$$ LB \approx 26 $$
$$ UB \approx 296 $$

Right away, we can see that the lower bound in this method is __far lower__ than the lower bound in the Tukey's Fences method and that the upper bound in this method is __far higher__ than the Tukey's Fences Method. This is most likely due to the relationship between our data. Since the data values are highly similar, our IQR is very small resulting in less extreme bounds in the Tukey's Fences method. In contrast, the highly similar data results in a very high standard deviation making much more extreme lower and upper bounds in this method.

This is reflected by the fact that only 2 students (Owain and Alaya) had values low enough to be considered outliers whereas in the Tukey's Fences method, four students were considered outliers.


### __Experiment 6__ : Define a function tardy_iqr().

In [47]:
def tardy_iqr(week):
    names = []
    week_list = []    
    with open('participants.csv', newline='') as csvfile:
        participant_rdr = csv.DictReader(csvfile, delimiter=',', quotechar='|')
        for row in participant_rdr:
            names.append(row['Student Name'])
            week_list.append(int(row[week]))
    quartile_vals = st.quantiles(week_list)
    q1 = quartile_vals[0]
    median = quartile_vals[1]
    q3 = quartile_vals[2]
    iqr = q3-q1
    k = 1.5
    lb = q1 - iqr * k
    ub = q3 + iqr * k
    indices = [index for index, element in enumerate(week_list) if element < lb or element > ub]
    out_names = [names[x] for x in indices]
    return out_names

print(f"Week 1 Outliers: {tardy_iqr('Week 1')}")

Week 1 Outliers: ['Adrian Ellison', 'Tayla Sparrow', 'Owain Emerson', 'Alaya Dickinson']


Experiment 6 focuses on performing the same task as Experiment 4: identifying the outliers of a dataset using the Tukey's Fences method. However, the difference is that Experiment 6 defines a function called tardy_iqr() that takes the name of a week (e.g., "Week 1", "Week 2", etc.) as a parameter. By doing so, the function can find the outliers of any week in the dataset. 

Additionally, this function optimizes the code of Experiment 4 by using list comprehension to identify the outliers in the dataset instead of populating a dictionary. The final output only consists of the names of all outlier students rather than both the names and values associated with them for that particular week.

To test the accuracy of this function, we ran the function using "Week 1" as a parameter and compared the results with the output of Experiment 4. Both outputs consist of the same outliers.

### __Experiment 7__ : Define a function tardy_stdev().

In [48]:
def tardy_stdev(week):
    names = []
    week_list = []
    with open('participants.csv', newline='') as csvfile:
        participant_rdr = csv.DictReader(csvfile, delimiter=',', quotechar='|')
        for row in participant_rdr:
            names.append(row['Student Name'])
            week_list.append(int(row[week]))
    standard_dev = st.stdev(week_list)
    mean = st.mean(week_list)
    lb = mean - 3*standard_dev  # define lower bound
    ub = mean + 3*standard_dev  # define upper bound
    # indicies contains a list of index values where there are outlier values in the week_list
    indices = [index for index, element in enumerate(week_list) if element < lb or element > ub]
    # now that we have indicies we can simply use them to select the outlier student names
    out_names = [names[x] for x in indices]
    return out_names

print(f"Week 1 Outliers: {tardy_stdev('Week 1')}")

Week 1 Outliers: ['Owain Emerson', 'Alaya Dickinson']


This experiment creates a function with the methodology described in Experiment 5. This method tardy_stdev() takes in the week's name (e.g., "Week 1", "Week 2", etc.) as a parameter and will find the outliers in the given week. While we did use the same methodology, we made optimizations to the code. The first design choice was to use list comprehension instead of for-loops to retrieve the outliers. Since we are only interested in returning the student names and not the outlier values, we did not use extra memory to populate a dictionary. Instead, we only used an array of indices to help us return the outlier student names. We ran the method on "Week 1" and verified that the output matched the results of Experiment 5.

### __Experiment 8__ : Compare tardy_iqr() and tardy_stdev() for Weeks 2-5.

In [49]:
for x in range (2,6):
    week = 'Week ' + str(x)
    print(f'Results for {week}')
    print(f"Outliers using Tukey's Fences: {tardy_iqr(week)}")
    print(f'Outliers using Standard Deviations: {tardy_stdev(week)} \n')

Results for Week 2
Outliers using Tukey's Fences: ['Yasir Fenton', 'Tamara Cottrell', 'Jazmin Foreman', 'Bear Zuniga', 'Miles Lyons', 'Owain Emerson']
Outliers using Standard Deviations: ['Miles Lyons', 'Owain Emerson'] 

Results for Week 3
Outliers using Tukey's Fences: ['Adrian Ellison', 'Adeline Jordan', 'Jaye Sweeney']
Outliers using Standard Deviations: ['Adrian Ellison'] 

Results for Week 4
Outliers using Tukey's Fences: ['Dora Delacruz', 'Shaquille Wood']
Outliers using Standard Deviations: ['Dora Delacruz'] 

Results for Week 5
Outliers using Tukey's Fences: ['Jazmin Foreman', 'Sanjay Edwards', 'Alfie-James Pierce', 'Adeline Jordan', 'Saffa Brook']
Outliers using Standard Deviations: ['Jazmin Foreman'] 



These results show the difference in outliers using both Tukey's Fences and the standard deviation method for weeks 2-5. These results reinforce the observations made in Experiment 5. Using the standard deviation method results in fewer outliers than the Tukey's Fences method due to the lower bound being much lower and the upper bound being much higher than the Tukey's Fences method. One method is not necessarily superior to another; it depends heavily on the dataset being analyzed. In this case, the data values are very similar since it models the amount of time a student is in class.

In most cases, this amount of time varies only slightly across multiple students. Because of this, our standard deviation is very high, but our IQR is very small. It makes more sense to use Tukey's Fences to look for outliers in this specific case because the standard deviation method is far too extreme to provide proper outlier detection.